In [3]:
import pandas as pd
import os, warnings, time, re, zipfile, requests, shutil
from bs4 import BeautifulSoup as bs

In [4]:
path = 'C:\\Users\\XXXXXXXX\\XXXXXXXX\\XXXXXXXX'

folders_list = os.listdir(path)

## Web Scrapping
Download de arquivos .zip contidos na página da Web fornecida.

In [ ]:
input_url = input('Type here the url of the page to be scrapped: ')

In [23]:
page = requests.get(input_url)
soup = bs(page.content, "html.parser")

In [ ]:
# listando todas as urls da página que contém um arquivo .zip para download

urls_list = list()

for link in soup.find_all('a', href=re.compile(".zip$")):
    hyperlink = link.get('href')
    urls_list.append(f'{input_url}/{hyperlink}')

In [32]:
# function that downloads a file in a given url

def download_file(url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename

In [ ]:
start = time.time()

for i in range(0, len(urls_list)):
    download_file(urls_list[i])
    print(f'Download do arquivo no link: {urls_list[i]} feito com sucesso! \n')
    
end = time.time() - start
duration = time.strftime("%H:%M:%S",time.gmtime(end))
print(f'Download dos arquivos finalizado. \nTempo de execução: {duration}.')

Download of file in http://200.152.38.155/CNPJ/Cnaes.zip: successfull 

Download of file in http://200.152.38.155/CNPJ/Empresas0.zip: successfull 

Download of file in http://200.152.38.155/CNPJ/Empresas1.zip: successfull 

Download of file in http://200.152.38.155/CNPJ/Empresas2.zip: successfull 

Download of file in http://200.152.38.155/CNPJ/Empresas3.zip: successfull 

Download of file in http://200.152.38.155/CNPJ/Empresas4.zip: successfull 

Download of file in http://200.152.38.155/CNPJ/Empresas5.zip: successfull 

Download of file in http://200.152.38.155/CNPJ/Empresas6.zip: successfull 



## Extração dos arquivos .zip

In [145]:
# função que extrai cada .zip em uma pasta de mesmo nome

start = time.time()

def unzip_batch(path):
    for root, dirs, files in os.walk(path, topdown=False):
        zip_files = list(filter(lambda file: file.endswith('.zip'), folders_list))
        for file in zip_files:
            file_path = f'{path}\\{file}'
            folder_name = file_path.replace('.zip', '')
            with zipfile.ZipFile(file_path,"r") as zip_ref:
                zip_ref.extractall(folder_name)
                print(f'O arquivo {file} foi descompactado com sucesso! \n')

In [ ]:
unzip_batch(path)

end = time.time() - start
duration = time.strftime("%H:%M:%S",time.gmtime(end))
print(f'Extração dos arquivos finalizada. \nTempo de execução: {duration}.')

The file Cnaes.zip was extracted with success! 

The file Dados_Abertos.zip was extracted with success! 

The file Empresas0.zip was extracted with success! 

The file Empresas1.zip was extracted with success! 

The file Empresas2.zip was extracted with success! 

The file Empresas3.zip was extracted with success! 



## Renomeando os arquivos .csv

In [60]:
start = time.time()

for root, dirs, files in os.walk(path, topdown=False):
    for folder in dirs:
        folder_path = os.path.join(root, folder)
        files = os.listdir(folder_path)
        files_filter = list(filter(lambda file: file.endswith('CSV') or file.endswith('ESTABELE'), files))
        #print(files_filter)
        for file in files_filter:
            old_file_path = f'{folder_path}\\{file}'
            new_file_path = f'{folder_path}\\{folder}.csv'
            os.rename(old_file_path, new_file_path)
            print(f'O arquivo {file} foi renomeado com sucesso! \n')
            
end = time.time() - start
duration = time.strftime("\n%H:%M:%S",time.gmtime(end))
print(f'Todos os arquivos foram renomeados. \nTempo de execução: {duration}.')

Extração dos arquivos finalizada. 
Tempo de execução: 
00:00:00.


## Agrupando arquivos

In [ ]:
new_folders = ['Empresas', 'Estabelecimentos', 'Socios']

In [ ]:
# Criando novas pastas para agrupar os arquivos relacionados

for name in new_folders:
    if not os.path.exists(f'{path}\\{name}'):
        os.makedirs(f'{path}\\{name}')

In [ ]:
def move_batch(folders_list):
        for root, dirs, files in os.walk(path, topdown=False):
            for file in files:
                for name in new_folders:
                    grouped_files = list(filter(lambda file: re.match(name, file) and file.endswith('.csv'), files))
                    for file in grouped_files:
                        old_folder_path = os.path.join(root, file)
                        new_folder_path = f'{path}\\{name}\\{file}'
                        shutil.move(old_folder_path, new_folder_path)
            print(f'O arquivo {file} foi movido com sucesso! \n')

In [ ]:
move_batch(new_folders)

In [ ]:
# removendo pastas vazias em que os .csv foram extraídos

for name in new_folders:
    for i in range(0, 10):
        if os.path.exists(f'{path}\\{name}{i}'):
            shutil.rmtree(f'{path}\\{name}{i}')
            print(f'A pasta vazia {name}{i} foi removida com sucesso! \n')

## Transformando .csv em .parquet

In [ ]:
input_folder = input('Insira o nome da pasta que deseja acessar: ')

In [ ]:
# definindo nome das colunas de cada arquivo

warnings.filterwarnings("ignore") 

colunas = {
    'Empresas' : ['cnpj_basico', 'razao_social', 'natureza_juridica', 'qualificacao_responsavel', 'capital_social',
                  'porte_empresa', 'ente_federativo_responsavel'],
    
    'Estabelecimentos' : ['cnpj_basico','cnpj_ordem','cnpj_dv','identificador_matriz_filial','nome_fantasia',
                          'situacao_cadastral','data_situacao_cadastral','motivo_situacao_cadastral',
                          'nome_cidade_exterior','pais','data_inicio_atividade','cnae_fiscal_principal',
                          'cnae_fiscal_secundaria','tipo_logradouro','logradouro','numero','complemento',
                          'bairro','cep','uf','municipio','ddd_1','telefone_1','ddd_2','telefone_2','ddd_fax',
                          'fax','correio_eletronico','situacao_especial','data_situacao_especial'],
    
    'Socios' : ['cnpj_basico','identificador_socio','nome_socio_razao_social','cpf_cnpj_socio','qualificacao_socio',
                'data_entrada_sociedade','pais','representante_legal','nome_do_representante',
                'qualificacao_representante_legal','faixa_etaria'],
    
    'Simples' : ['cnpj_basico','opcao_pelo_simples','data_opcao_simples','data_exclusao_simples','opcao_mei',
                 'data_opcao_mei','data_exclusao_mei'],
    
    'Cnaes': ['codigo', 'descricao'],
    
    'Motivos': ['codigo', 'descricao'],
    
    'Municipios' : ['codigo', 'descricao'],
    
    'Naturezas' : ['codigo', 'descricao'],
    
    'Paises' : ['codigo', 'descricao'],
    
    'Qualificacoes': ['codigo', 'descricao']
}

In [ ]:
start = time.time()

input_path = f'{path}\\{input_folder}'
os.listdir(input_path)

In [8]:
for root, dirs, files in os.walk(input_path, topdown=False):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            
            coluna_temp = colunas.get(re.sub(r'[0-9]', '',file.replace('.csv',''))) #Colunas para o df, conforme o arquivo
            df = pd.read_csv(file_path, sep=';',
                             skiprows=0,
                             header=None,
                             encoding= "ISO-8859-1",
                             dtype = 'object')
            df = df.where(df.notnull(), 'N/A')
            df.columns = df.columns.astype(str)
            df.columns = coluna_temp
            df_min = df.head(5)
            
            parquet_path = file_path.replace('.csv', '.parquet')
            parquet_min_path = parquet_path.replace('.parquet', '_min.parquet')
            
            if os.path.exists(parquet_path):
                os.remove(parquet_path)
            else:
                pass
            
            if os.path.exists(parquet_min_path):
                os.remove(parquet_min_path)
            else:
                pass
            
            df.to_parquet(parquet_path)
            df_min.to_parquet(parquet_min_path, index=False)
            
            print(f'Conversão do arquivo {file} para parquet bem sucedida!')
            print(f'Criação do arquivo {file.replace('.csv', '_min.parquet')} bem sucedida \n')
            print('')

end = time.time() - start
duration = time.strftime("%H:%M:%S",time.gmtime(end))
print(f'Finalizada a geração dos arquivos.\nTempo de execução: {duration}.')

Conversion of the file Estabelecimentos0.csv to parquet: successful
Creation of the sample file C:\Users\menezebe\OneDrive - Merck Sharp & Dohme LLC\Documents\db_cnpj\data_renamed\Estabelecimentos\Estabelecimentos0_min.parquet: successful

Conversion of the file Estabelecimentos1.csv to parquet: successful
Creation of the sample file C:\Users\menezebe\OneDrive - Merck Sharp & Dohme LLC\Documents\db_cnpj\data_renamed\Estabelecimentos\Estabelecimentos1_min.parquet: successful

Conversion of the file Estabelecimentos2.csv to parquet: successful
Creation of the sample file C:\Users\menezebe\OneDrive - Merck Sharp & Dohme LLC\Documents\db_cnpj\data_renamed\Estabelecimentos\Estabelecimentos2_min.parquet: successful

Conversion of the file Estabelecimentos3.csv to parquet: successful
Creation of the sample file C:\Users\menezebe\OneDrive - Merck Sharp & Dohme LLC\Documents\db_cnpj\data_renamed\Estabelecimentos\Estabelecimentos3_min.parquet: successful

Conversion of the file Estabelecimentos4

In [ ]:
# checando se todos os arquivos foram devidamente criados

for folder in folders_list:
    files = os.listdir(f'{path\}\\{folder}')
    print(f'Folder: {folder}')
    print('------------------------------------------')
    for file in files:
        print(file)
    print('')

In [9]:
# isso aqui é apenas para eu excluir alguns arquivos que criei errados

for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        if file.endswith('.parquet_min'):
            file_path = os.path.join(root, file)
            os.remove(file_path)
            print(f'File {file} removed with success!')
        else:
            print("This file is OK.")

This file is OK.
This file is OK.
File Motivos.parquet_min removed with success!
